### Importando bibliotecas necessárias

In [281]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Lendo dados da planilha

In [282]:
"RES. ITUPARARANGA (próx. a Barragem)"
data = pd.read_excel('./Dados_ResItupararanga/dados.xlsx')
dataNotas = data.copy()
dataNotas.head(4)

,Data,TempAmostra,pH,Turbidez,DBO,OD,SolTotais,ColiformesFecais,FosforoTotal,NitrogenioTotal,IQA
0,2010-11-18,21.8,7.3,6.1,2,8.1,42.0,1,0.03,0.86,92
1,2010-09-23,21.1,6.9,5.1,6,7.9,88.0,1,0.02,0.50,88
2,2010-07-27,21.2,7.5,7.3,2,8.6,80.0,13,0.09,0.55,84
3,2010-05-20,20.5,6.7,2.5,2,6.4,56.0,1,0.02,0.58,90


### Cálculo da nota de OD (Oxigênio Dissolvido) [% de saturação]

Os dados de OD vieram em mg/L, portanto é necessário fazer a coversão.

In [283]:
altitude = 840
EqGeralOD = 14.62 - (0.3898 * dataNotas["TempAmostra"]) +\
             (0.006969 * (dataNotas["TempAmostra"] ** 2) -\
              (0.00005896 * (dataNotas["TempAmostra"] ** 3))) * ((1 - 0.0000228675 * altitude)) ** 5,167

OD_saturado = dataNotas["OD"] * 100 / EqGeralOD[0]

i = 0
resultadoOD = []

for i in range(len(OD_saturado)):
    
    if OD_saturado[i] > 140:
        resultadoOD.append(47)
        
    elif OD_saturado[i] > 100:
        resultadoOD.append(((2.9 * OD_saturado[i])- (0.02496 * OD_saturado[i] ** 2) +\
                            (5.60919 * 0.00001 * (OD_saturado[i] ** 3))) + 3)
        
    elif OD_saturado[i] > 85:
        resultadoOD.append((3.7745 * (OD_saturado[i] ** 0.704889)) + 3)
        
    elif OD_saturado[i] > 50:
        resultadoOD.append(((-1.166 * OD_saturado[i]) + (0.058 * (OD_saturado[i] ** 2)) -\
                             (3.803435 * 0.0001 * (OD_saturado[i] ** 3))) + 3)
        
    else:
        resultadoOD.append(((0.34 * OD_saturado[i]) + (0.008095 * (OD_saturado[i] ** 2)) +\
                             (1.35252 * 0.00001 * (OD_saturado[i] ** 3))) + 3)
        
dataNotas["OD"] = resultadoOD

### Cálculo da nota de Temperatura da Amostra [°C]

Considera-se que a diferença de temperatura é zero, uma vez que nossos
cursos hídricos não apresentam grande diferença a montante e a jusante do ponto
escolhido, sendo assim o subíndice é considerado igual a 94. 

In [284]:
dataNotas["TempAmostra"] = 94

### Cálculo da nota de Coliformes [NPM/100mL]

In [285]:
i = 0
resultadoColiformes = []

for i in range(len(dataNotas["ColiformesFecais"])):
    
    if dataNotas["ColiformesFecais"][i] == 0:
        resultadoColiformes.append(100)
        
    else:
        coli10 = np.log10(dataNotas["ColiformesFecais"][i])
        
        if coli10 > 5:
            resultadoColiformes.append(3)
            
        elif coli10 > 1:
            resultadoColiformes.append(100 - (37.2 * coli10) + (3.60743 * (coli10 ** 2)))
            
        else:
            resultadoColiformes.append(100 - (33 * coli10))

dataNotas["ColiformesFecais"] = resultadoColiformes

### Cálculo da nota de pH

In [286]:
i = 0
resultadopH = []

for i in range(len(dataNotas["pH"])):
    
    if dataNotas["pH"][i] <= 2:
        resultadopH.append(2)
        
    elif dataNotas["pH"][i] <= 4:
        resultadopH.append(13.6 - (10.6 * dataNotas["pH"][i]) + (2.4364 * (dataNotas["pH"][i] ** 2)))
        
    elif dataNotas["pH"][i] <= 6.2:
        resultadopH.append(155.5 - (77.36 * dataNotas["pH"][i]) + (10.2481  * (dataNotas["pH"][i] ** 2)))
        
    elif dataNotas["pH"][i] <= 7:
        resultadopH.append(-657.2 + (197.38 * dataNotas["pH"][i]) - (12.9167  * (dataNotas["pH"][i] ** 2)))
        
    elif dataNotas["pH"][i] <= 8:
        resultadopH.append(-427.8 + (142.05 * dataNotas["pH"][i]) - (9.695  * (dataNotas["pH"][i] ** 2)))
    
    elif dataNotas["pH"][i] <= 8.5:
        resultadopH.append(216 - (16 * dataNotas["pH"][i]))
        
    elif dataNotas["pH"][i] <= 9:
        resultadopH.append(1415823 * (np.exp(-1.1507 * dataNotas["pH"][i])))
        
    elif dataNotas["pH"][i] <= 10:
        resultadopH.append(228 - (27 * dataNotas["pH"][i]))
        
    elif dataNotas["pH"][i] <= 12:
        resultadopH.append(633 - (106.5 * dataNotas["pH"][i]) + (4.5  * (dataNotas["pH"][i] ** 2)))
        
    else:
        resultadopH.append(3)

dataNotas["pH"] = resultadopH

### Cálculo da nota de DBO (Demanda Bioquímica de Oxigênio) [mg/L]

In [287]:
i = 0
resultadoDBO = []

for i in range(len(dataNotas["DBO"])):
    
    if dataNotas["DBO"][i] <= 5:
        resultadoDBO.append(99.96 * np.exp((-0.1232728) * dataNotas["DBO"][i]))
    
    elif dataNotas["DBO"][i] <= 15:
        resultadoDBO.append(104.67 - (31.5463 * np.log10(dataNotas["DBO"][i])))
        
    elif dataNotas["DBO"][i] <= 30:
        resultadoDBO.append(4394.91 * (dataNotas["DBO"][i] ** (-1.99809)))
        
    else:
        resultadoDBO.append(2)
        
dataNotas["DBO"] = resultadoDBO

### Cálculo da nota de Nitrogênio Total [mg/L]

In [288]:
i = 0
resultadoNT = []

for i in range(len(dataNotas["NitrogenioTotal"])):  
    
    if dataNotas["NitrogenioTotal"][i] <= 10:
        resultadoNT.append(100 - (8.169 * dataNotas["NitrogenioTotal"][i]) +\
                           (0.3059 * (dataNotas["NitrogenioTotal"][i] ** 2)))
    
    elif dataNotas["NitrogenioTotal"][i] <= 60:
        resultadoNT.append(101.9 - 23.1023 * np.log(dataNotas["NitrogenioTotal"][i]))
        
    elif dataNotas["NitrogenioTotal"][i] <= 100:
        resultadoNT.append(159.3148 * 2.7828 ** (dataNotas["NitrogenioTotal"][i] * (-0.512842)))
        
    else:
        resultadoNT.append(1)
        
dataNotas["NitrogenioTotal"] = resultadoNT

### Cálculo da nota de Fósforo Total [mg/L]

É necessário converter para fosfato. O fator de conversão é 3.066

In [289]:
i = 0
resultadoFT = []

for i in range(len(dataNotas["FosforoTotal"])):
    PO4 = dataNotas["FosforoTotal"][i] * 3.066
    
    if dataNotas["FosforoTotal"][i] <= 1:
        resultadoFT.append(99 * np.exp((-0.91629 * PO4)))
    
    elif dataNotas["FosforoTotal"][i] <= 5:
        resultadoFT.append(57.6 - 20.178 * PO4 + 2.1326 * (PO4 ** 2))
        
    elif dataNotas["FosforoTotal"][i] <= 10:
        resultadoFT.append(19.8 * np.exp((-0.13544 * PO4)))
        
    else:
        resultadoFT.append(1)
        
dataNotas["FosforoTotal"] = resultadoFT

### Cálculo da nota de Turbidez [UFT]

In [290]:
i = 0
resultadoTurbidez = []

for i in range(len(dataNotas["Turbidez"])):
    
    if dataNotas["Turbidez"][i] <= 25:
        resultadoTurbidez.append(100.17 - 2.67 * dataNotas["Turbidez"][i] + 0.03775 * (dataNotas["Turbidez"][i] ** 2))
    
    elif dataNotas["Turbidez"][i] <= 100:
        resultadoTurbidez.append(84.76 * np.exp(-0.016206 * dataNotas["Turbidez"][i]))
        
    else:
        resultadoTurbidez.append(5)
        
dataNotas["Turbidez"] = resultadoTurbidez

### Cálculo da nota de Sólidos Totais [mg/L]

In [291]:
i = 0
resultadoST = []

for i in range(len(dataNotas["SolTotais"])):
    
    if dataNotas["SolTotais"][i] <= 150:
        resultadoST.append(79.75 + 0.166 * dataNotas["SolTotais"][i] - 0.001088 * (dataNotas["SolTotais"][i] ** 2))
    
    elif dataNotas["SolTotais"][i] <= 500:
        resultadoST.append(101.67 - 0.13917 * dataNotas["SolTotais"][i])
        
    else:
        resultadoST.append(32)
        
dataNotas["SolTotais"] = resultadoST

### Cálculo do IQA

In [292]:
pesos = {
    "OD" : 0.17,
    "ColiformesFecais" : 0.15,
    "pH" : 0.12,
    "DBO" : 0.1,
    "FosforoTotal" : 0.1,
    "TempAmostra" : 0.1,
    "NitrogenioTotal" : 0.1,
    "Turbidez" : 0.08,
    "SolTotais" : 0.08
}

dataNotas["IQA"] = 0
for key in pesos:
    dataNotas["IQA"] += dataNotas[key] * pesos[key]
    
dataNotas.head(4)

,Data,TempAmostra,pH,Turbidez,DBO,OD,SolTotais,ColiformesFecais,FosforoTotal,NitrogenioTotal,IQA
0,2010-11-18,94,92.518450,85.287678,78.118313,96.161000,84.802768,100.000000,90.998179,93.200904,91.688559
1,2010-09-23,94,89.757913,87.534877,80.122207,93.533821,85.932528,100.000000,93.590880,95.991975,91.919598
2,2010-07-27,94,92.231250,82.690697,78.118313,99.268462,86.066800,63.037658,76.882539,95.599585,85.359681
3,2010-05-20,94,85.415337,93.730938,78.118313,78.401747,85.634032,100.000000,93.590880,95.364885,89.034743


In [295]:
dataNotas.to_excel("./Dados_ResItupararanga/IQA_Calculado.xlsx")